Multi-arm bandits
Multi-arm bandits are a class of problems in which fixed resources must be allocated to different decisions to maximize a return value. Consider a casino with 10 different slot machines to choose from. Assume that each slot machine has an unknown and potentially different mean reward and that you get a random reward (positive or negative) when you play it.

To maximize the return in this game, you need to explore all machines to get a sense of reward values. But you do not want to greedily stick to a machine if it returns a high reward once. In other words, you need to explore and gain confidence in the average reward for each slot, which can be achieved using an epsilon-greedy selection strategy.

In [ ]:
# required imports

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
%matplotlib inline

STEP_SIZE = 0.1
K = 4

In [ ]:
def epsilon_greedy_selection(epsilon, Q):
    if np.random.rand() < epsilon:
        # Exploration: choose a random action
        return np.random.randint(len(Q))
    
    # Exploitation: choose the action with the highest Q-value
    max_value = np.max(Q)
    max_indices = np.where(Q == max_value)[0]  # get all indices with max Q
    return np.random.choice(max_indices)       # break ties randomly

In [ ]:
# sanity check
rewards = np.array([3, 2, 1])
# pure greedy
for _ in range(100):
    assert epsilon_greedy_selection(0, rewards) == 0
    
# pure greedy with more than one max reward.
rewards = np.array([3, 3, 1, 2])
for _ in range(100):
    index = epsilon_greedy_selection(0, rewards)
    assert index == 0 or index == 1

In [ ]:
# allow some stochasiticity
epsilon = 0.1
indices = [epsilon_greedy_selection(epsilon, rewards) for _ in range(1000)]
obtained_rewards = [rewards[i] for i in indices]
obtained_rewards = np.array(obtained_rewards)
mean_reward = np.mean(obtained_rewards)
assert 2.5 <= mean_reward <= 3

We have implemented a helper class for you that simulates playing slot machines 1000 times for different values of epsilon. Note that  𝜖=0
  would correspond to a pure greedy strategy while  𝜖=1
  would correspond to a pure random strategy.

In your implementation of epsilon_greedy_selection, you should be able to observe that using  𝜖=0.01
  gives a better reward in the long run than using  𝜖=0
 .

In [ ]:
class SlotMachine:
    def __init__(self, eps):
        self.epsilon = eps
        
    def reset(self):
        # real reward for each action
        self.q_true = np.random.randn(K)

        # estimation for each action
        self.q_estimation = np.zeros(K)
        
        self.best_action = np.argmax(self.q_true)
   
    # take an action, update estimation for this action
    def step(self, action):
        # generate the reward under N(real reward, 1)
        reward = np.random.randn() + self.q_true[action]

        # update estimation with constant step size
        self.q_estimation[action] += STEP_SIZE * (reward - self.q_estimation[action])
        return reward


def simulate(runs, time, slotMachines):
    rewards = np.zeros((len(slotMachines), runs, time))
    for i, machine in enumerate(slotMachines):
        for r in tqdm(range(runs)):
            machine.reset()
            for t in range(time):
                
                # use epislon greedy selection
                action = epsilon_greedy_selection(machine.epsilon, machine.q_estimation)
                
                reward = machine.step(action)
                rewards[i, r, t] = reward
    mean_rewards = rewards.mean(axis=1)
    return mean_rewards


runs=400
time=1000
epsilons = [0.0, 0.01]
slotMachines = [SlotMachine(eps) for eps in epsilons]
rewards_new = simulate(runs, time, slotMachines)

plt.figure(figsize=(10, 10))
for eps, rewards_new in zip(epsilons, rewards_new):
    plt.plot(rewards_new, label='$\epsilon = %.02f$' % (eps))
plt.xlabel('steps')
plt.ylabel('average reward')
plt.legend()
plt.show()